In [3]:
import numpy as np
import pandas as pd

In [149]:
# data = pd.read_csv('../data/raw/ydata-ynacc-v1_0_expert_annotations.tsv', sep='\t')
data = pd.read_csv('../data/datasets/German/raw.csv', sep='\t')
german_data = pd.read_csv('../data/datasets/German/raw_de.csv', sep='\t')

In [117]:
grouped_data = data.groupby(['commentid'])
def has_label(comment, column, label):
    return np.sum(grouped_data.get_group(comment)[column].apply(lambda x: str(x).find(label) > -1)) > 0

In [118]:
data['y_persuasive'] = data.apply(lambda x: has_label(x['commentid'], 'persuasiveness', 'Persuasive', axis=1)
data['y_audience'] = data.apply(lambda x: has_label(x['commentid'], 'intendedaudience', 'Reply to a specific commenter', axis=1)
data['y_agreement_with_commenter'] = data.apply(lambda x: has_label(x['commentid'], 'commentagreement', 'Agreement with commenter', axis=1)
data['y_informative'] = data.apply(lambda x: has_label(x['commentid'], 'tone', 'Informative', axis=1)
data['y_mean'] = data.apply(lambda x: has_label(x['commentid'], 'tone', 'Mean', axis=1)
data['y_controversial'] = data.apply(lambda x: has_label(x['commentid'], 'tone', 'Controversial', axis=1)
data['y_disagreement_with_commenter'] = data.apply(lambda x: has_label(x['commentid'], 'commentagreement', 'Disagreement with commenter', axis=1)
data['y_off_topic_with_article'] = data.apply(lambda x: has_label(x['commentid'], 'topic', 'Off-topic with article', axis=1)
data['y_sentiment_neutral'] = data.apply(lambda x: has_label(x['commentid'], 'sentiment', 'neutral', axis=1)
data['y_sentiment_positive'] = data.apply(lambda x: has_label(x['commentid'], 'sentiment', 'positive', axis=1)
data['y_sentiment_negative'] = data.apply(lambda x: has_label(x['commentid'], 'sentiment', 'negative', axis=1)
data['y_sentiment_mixed'] = data.apply(lambda x: has_label(x['commentid'], 'sentiment', 'mixed', axis=1)

In [ ]:
# data['y_persuasive'] = data['persuasiveness'].str.contains('Persuasive')
# data['y_audience'] = data['intendedaudience'].str.contains('Reply to a specific commenter')
# data['y_agreement_with_commenter'] = data['commentagreement'].str.contains('Agreement with commenter')
# data['y_informative'] = data['tone'].str.contains('Informative')
# data['y_mean'] = data['tone'].str.contains('Mean')
# data['y_controversial'] = data['tone'].str.contains('Controversial')
# data['y_disagreement_with_commenter'] = data['commentagreement'].str.contains('Disagreement with commenter')
# data['y_off_topic_with_article'] = data['topic'].str.contains('Off-topic with article')
# data['y_sentiment_neutral'] = data['sentiment'].str.contains('neutral')
# data['y_sentiment_positive'] = data['sentiment'].str.contains('positive')
# data['y_sentiment_negative'] = data['sentiment'].str.contains('negative')
# data['y_sentiment_mixed'] = data['sentiment'].str.contains('mixed')

In [39]:
tiny = data.drop_duplicates(['y_persuasive', 'y_audience', 'y_agreement_with_commenter', 'y_informative', 'y_mean', 'y_controversial', 'y_disagreement_with_commenter', 'y_off_topic_with_article', 'y_sentiment_neutral', 'y_sentiment_positive', 'y_sentiment_negative', 'y_sentiment_mixed'])

In [72]:
def valid_dataset(df):
    return np.sum(df['y_persuasive']) >= 1 and np.sum(df['y_audience']) >= 1 and np.sum(df['y_agreement_with_commenter']) >= 1 and np.sum(df['y_informative']) >= 1 and np.sum(df['y_mean']) >= 1 and np.sum(df['y_controversial']) >= 1 and np.sum(df['y_disagreement_with_commenter']) >= 1 and np.sum(df['y_off_topic_with_article']) >= 1 and np.sum(df['y_sentiment_neutral']) >= 1 and np.sum(df['y_sentiment_positive']) >= 1 and np.sum(df['y_sentiment_negative']) >= 1 and np.sum(df['y_sentiment_mixed']) >= 1

In [73]:
tmp = tiny.sample(10)
while not valid_dataset(tmp):
    tmp = tiny.sample(10)

In [77]:
tmp = tmp.fillna(False)
tmp.to_csv('../data/datasets/Tiny/train/comments.csv', index=False)
tmp.to_csv('../data/datasets/Tiny/test/comments.csv', index=False)

In [201]:
# save_data = data.fillna(False).drop_duplicates(['commentid'])
save_data.to_csv('../data/datasets/YNACC/train/comments.csv', index=False)
save_data.to_csv('../data/datasets/YNACC/test/comments.csv', index=False)

In [174]:
# save user meta data
users_grouped = save_data.groupby('guid')
user_data = {}
for name, group in users_grouped:
    threads_participated_in = len(np.unique(group['parentid'].apply(lambda x: str(x))))
    user_data[name] = {
        'comment_count': len(group),
        'threads_participated_in': threads_participated_in,
        'threads_initiated': np.sum(group['parentid'] == False),
        'tu_received': np.sum(group['thumbs-up']),
        'td_received': np.sum(group['thumbs-down']),
        'commenting_rate': (np.max(group['timestamp']) - np.min(group['timestamp'])) / threads_participated_in
    }
    
long_data = pd.read_csv('../data/raw/ydata-ynacc-v1_0_unlabeled_conversations.csv', sep=';')
save_long_data = long_data.fillna(False).drop_duplicates(['commentid'])
long_users_grouped = save_long_data.groupby('guid')
for name, group in long_users_grouped:
    threads_participated_in = len(np.unique(group['parentid'].apply(lambda x: str(x))))
    if name in user_data:
        data = user_data[name]
        data['comment_count'] = data['comment_count'] + len(group)
        data['threads_participated_in'] = data['threads_participated_in'] + threads_participated_in
        data['threads_initiated'] = data['threads_initiated'] + np.sum(group['parentid'] == False)
        data['tu_received'] = data['tu_received'] + np.sum(group['thumbs-up'])
        data['td_received'] = data['td_received'] + np.sum(group['thumbs-down'])
        data['commenting_rate'] = (data['commenting_rate'] + (np.max(group['timestamp']) - np.min(group['timestamp'])) / threads_participated_in)/2
        user_data[name] = data

users = pd.DataFrame.from_dict(user_data, orient='index').reset_index()
users.to_csv('../data/datasets/YNACC/users.csv', index=False)

### YNACC Train/Dev/Test Sets

In [202]:
train_ids = np.array(open('../data/raw/ydata-ynacc-v1_0_train-ids.txt', 'r').read().splitlines()).astype('int64')
dev_ids = np.array(open('../data/raw/ydata-ynacc-v1_0_dev-ids.txt', 'r').read().splitlines()).astype('int64')
test_ids = np.array(open('../data/raw/ydata-ynacc-v1_0_test-ids.txt', 'r').read().splitlines()).astype('int64')

In [203]:
save_data[save_data['sdid'].isin(train_ids)].to_csv('../data/datasets/YNACC-Evaluation/train/comments.csv', index=False)
save_data[save_data['sdid'].isin(test_ids)].to_csv('../data/datasets/YNACC-Evaluation/dev/comments.csv', index=False)
save_data[save_data['sdid'].isin(dev_ids)].to_csv('../data/datasets/YNACC-Evaluation/test/comments.csv', index=False)

# German Data

In [24]:
data = pd.read_csv('../data/datasets/German/raw.csv', sep=',')
german_data = pd.read_csv('../data/datasets/German/raw_de.csv', sep=';')

grouped_data = data.groupby(['commentid'])
def has_label(comment, column, label):
    return np.sum(grouped_data.get_group(comment)[column].apply(lambda x: str(x).find(label) > -1)) > 0

data['y_persuasive'] = data.apply(lambda x: has_label(x['commentid'], 'persuasiveness', 'Persuasive'), axis=1)
data['y_audience'] = data.apply(lambda x: has_label(x['commentid'], 'intendedaudience', 'Reply to a specific commenter'), axis=1)
data['y_agreement_with_commenter'] = data.apply(lambda x: has_label(x['commentid'], 'commentagreement', 'Agreement with commenter'), axis=1)
data['y_informative'] = data.apply(lambda x: has_label(x['commentid'], 'tone', 'Informative'), axis=1)
data['y_mean'] = data.apply(lambda x: has_label(x['commentid'], 'tone', 'Mean'), axis=1)
data['y_controversial'] = data.apply(lambda x: has_label(x['commentid'], 'tone', 'Controversial'), axis=1)
data['y_disagreement_with_commenter'] = data.apply(lambda x: has_label(x['commentid'], 'commentagreement', 'Disagreement with commenter'), axis=1)
data['y_off_topic_with_article'] = data.apply(lambda x: has_label(x['commentid'], 'topic', 'Off-topic with article'), axis=1)
data['y_sentiment_neutral'] = data.apply(lambda x: has_label(x['commentid'], 'sentiment', 'neutral'), axis=1)
data['y_sentiment_positive'] = data.apply(lambda x: has_label(x['commentid'], 'sentiment', 'positive'), axis=1)
data['y_sentiment_negative'] = data.apply(lambda x: has_label(x['commentid'], 'sentiment', 'negative'), axis=1)
data['y_sentiment_mixed'] = data.apply(lambda x: has_label(x['commentid'], 'sentiment', 'mixed'), axis=1)
                                       
german_data['y_persuasive'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_audience'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_agreement_with_commenter'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_informative'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_mean'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_controversial'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_disagreement_with_commenter'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_off_topic_with_article'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_sentiment_neutral'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_sentiment_positive'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_sentiment_negative'] = np.ones(german_data.shape[0], dtype=bool)
german_data['y_sentiment_mixed'] = np.ones(german_data.shape[0], dtype=bool)
                                       
data.to_csv('../data/datasets/German/train/comments.csv', index=False)                                
german_data.to_csv('../data/datasets/German/test/comments.csv', index=False)   